In [3]:
import pytest
import torch

from nerfacc import OccupancyGrid, ray_marching, unpack_info

import taichi as ti
ti.init(arch=ti.cuda)

ImportError: cannot import name 'NGPradianceField' from 'nerfacc' (/home/loyot/workspace/code/nerfacc/nerfacc/__init__.py)

In [2]:
device = "cuda:0"
batch_size = 128

In [3]:
def test_marching_with_grid_cuda():
    rays_o = torch.rand((batch_size, 3), device=device)
    rays_d = torch.randn((batch_size, 3), device=device)
    rays_d = rays_d / rays_d.norm(dim=-1, keepdim=True)
    grid = OccupancyGrid(roi_aabb=[0, 0, 0, 1, 1, 1]).to(device)
    grid._binary[:] = True

    packed_info, t_starts, t_ends = ray_marching(
        rays_o,
        rays_d,
        grid=grid,
        near_plane=0.0,
        far_plane=1.0,
        render_step_size=1e-2,
    )
    torch.cuda.synchronize()

In [7]:
%timeit test_marching_with_grid_cuda() # grid_binary.flatten() with block

13.8 ms ± 25.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [7]:
%timeit test_marching_with_grid_cuda() # packed_info.flatten() t_starts.flatten() t_ends.flatten()

14.5 ms ± 139 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [8]:
%timeit test_marching_with_grid_cuda() # no block

14.4 ms ± 90.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [8]:
%timeit test_marching_with_grid_cuda() # grid_binary shape 1

14.7 ms ± 325 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [7]:
%timeit test_marching_with_grid_cuda() #cuda

13.9 ms ± 66.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [9]:
import tinycudann as tcnn
import numpy as np

In [34]:
# constants
n_levels = 4
targe_resolution = 1024
base_resolution = 256
b = np.exp(np.log(targe_resolution/base_resolution)/(n_levels-1))
per_level_scale = b

posi_encoder = tcnn.Encoding(
    n_input_dims=3,
    encoding_config={
        "otype": "HashGrid",
        "n_levels": n_levels,
        "n_features_per_level": 2,
        "log2_hashmap_size": 19,
        "base_resolution": base_resolution,
        "per_level_scale": np.exp(np.log(4/2)/(8-1)),
    }
)

In [33]:
np.exp(np.log(4/2)/(8-1))

1.1040895136738123

In [29]:
np.exp(np.log(4/2)/(8-1))

1.1040895136738123

In [35]:
b_tcnn = posi_encoder.native_tcnn_module.hyperparams()['per_level_scale']
print(f"python per_level_scale: {b}")
print(f"tcnn per_level_scale: {b_tcnn}")

python per_level_scale: 1.5874010519681994
tcnn per_level_scale: 1.1040894985198975


In [37]:
ress = []
for i in range(8):
    ress.append(int(np.ceil(2 * np.exp(i*np.log(b_tcnn)) - 1.0)) + 1)
    
print(f"tcnn res: {ress}")

ress = []
for i in range(n_levels):
    ress.append(int(np.ceil(2 * np.exp(i*np.log(b)) - 1.0)) + 1)
    
print(f"python res: {ress}")

tcnn res: [2, 3, 3, 3, 3, 4, 4, 4]
python res: [2, 4, 6, 8]


In [14]:
posi_encoder.params.shape

torch.Size([12196240])

In [10]:
3*64+64*64+64*64+64*3

8576

In [70]:
2**21

2097152

In [77]:
38**4 - 2**21

-12016

In [29]:
26 / 2

13.0

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 150
from ipywidgets import interact, widgets
from PIL import Image
import glob

import re
def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    '''
    return [ atoi(c) for c in re.split(r'(\d+)', text) ]

In [2]:
def read_images(file_list):
    images = []
    for i in file_list:
        images.append(Image.open(i))
    
    return images
        
        
def browse_images(digits):
    n = len(digits)
    w = widgets.IntSlider(value=0, min=0, max=(n-1))
    def view_image(x):
        plt.imshow(digits[x], cmap=plt.cm.gray_r, interpolation='nearest')
        plt.title(f'rgb_{x}.png')
        plt.show()
    interact(view_image, x=w)

In [5]:
root = "/home/loyot/workspace/code/training_results/nerfacc/ngp_dnerf/trex/lr_0-01/pf_nopw_l-huber/training/rgb*.png"
images_list = glob.glob(root)
images_list.sort(key=natural_keys)
image_data = read_images(images_list)
browse_images(image_data)

interactive(children=(IntSlider(value=0, description='x', max=20), Output()), _dom_classes=('widget-interact',…

In [85]:
root = "/home/loyot/workspace/code/training_results/nerfacc/ngp_dnerf/trex/lr_0-01/pf_nopw_l-huber/training/rgb*.png"
images_list = glob.glob(root)
images_list.sort(key=natural_keys)
image_data = read_images(images_list)
browse_images(image_data)

interactive(children=(IntSlider(value=0, description='x', max=20), Output()), _dom_classes=('widget-interact',…

In [31]:
image_data = read_images(images_list)
browse_images(image_data)

interactive(children=(IntSlider(value=15, description='x', max=19), Output()), _dom_classes=('widget-interact'…